## Preprocessing

In [10]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split

In [11]:
data1 = pd.read_csv("D:/Github/analisis-sosial-media/azarine_Agt12.csv")
data2 = pd.read_csv("D:/Github/analisis-sosial-media/azarine_Agt810.csv")
data3 = pd.read_csv("D:/Github/analisis-sosial-media/azarine_Agt1516.csv")
data4 = pd.read_csv("D:/Github/analisis-sosial-media/azarine_Agt2223.csv")
data5 = pd.read_csv("D:/Github/analisis-sosial-media/azarine_Sep12.csv")
data6 = pd.read_csv("D:/Github/analisis-sosial-media/azarine_Sep810.csv")
data7 = pd.read_csv("D:/Github/analisis-sosial-media/azarine_Sep1516.csv")

# Menggabungkan semua dataframe secara vertikal
dataset = pd.concat([data1, data2, data3, data4, data5, data6, data7], ignore_index=True)

# Menampilkan hasil gabungan
print(dataset)

      conversation_id_str                      created_at  favorite_count  \
0     1819522350448083282  Fri Aug 02 23:55:08 +0000 2024               1   
1     1819282637896532132  Fri Aug 02 22:49:44 +0000 2024               0   
2     1819436611425419435  Fri Aug 02 18:14:26 +0000 2024               0   
3     1819434300607173043  Fri Aug 02 18:05:15 +0000 2024               0   
4     1819247269960691742  Fri Aug 02 17:55:38 +0000 2024               0   
...                   ...                             ...             ...   
1418  1835147140655669688  Sun Sep 15 02:42:28 +0000 2024               0   
1419  1835142127476785369  Sun Sep 15 02:22:33 +0000 2024               0   
1420  1835102249582051333  Sun Sep 15 02:10:57 +0000 2024               1   
1421  1835119763691127104  Sun Sep 15 01:57:58 +0000 2024               0   
1422  1834965594497753187  Sun Sep 15 01:50:24 +0000 2024               0   

                                              full_text               id_st

In [12]:
# dataset = pd.read_csv("C:/Users/yunit/Downloads/azarine.csv")

# Menghapus baris di mana kolom 'username' mengandung kata 'azarine'
dataset = dataset[~dataset['username'].str.contains('azarine', case=False, na=False)]

#Menghapus kolom yang tidak diperlukan
dataset = dataset.drop(columns=['conversation_id_str', 'id_str', 'image_url', 'location', 'tweet_url', 'user_id_str', 'in_reply_to_screen_name', 'lang'])

dataset

,created_at,favorite_count,full_text,quote_count,reply_count,retweet_count,username
0,Fri Aug 02 23:55:08 +0000 2024,1,ss azarine ini bikin kucel ga ya? kalau iya s...,0,22,0,baseconvo
1,Fri Aug 02 22:49:44 +0000 2024,0,@aceaurs Azarine kak anak ny temen ku usia rem...,0,1,0,yayangke
2,Fri Aug 02 18:14:26 +0000 2024,0,Aku nemu promo menarik di Lazada nih. Yuk coba...,0,0,0,disc_beauty
3,Fri Aug 02 18:05:15 +0000 2024,0,Kode Kecantikan ️ 3 Agu ⏰️ 00:00 sampai habis ...,1,0,0,AkulahPara
4,Fri Aug 02 17:55:38 +0000 2024,0,@JY15O1 NAHHH BEGO NYA AKU PAKE AZARINE JUGA I...,0,0,0,calovessa
...,...,...,...,...,...,...,...
1418,Sun Sep 15 02:42:28 +0000 2024,0,Cek [GEL] Azarine Hydrasoothe Sunscreen Gel SP...,0,0,0,Affilateshopee
1419,Sun Sep 15 02:22:33 +0000 2024,0,plss sunscreen azarine ini rekomend bgtt guyss...,0,0,0,rccchaa
1420,Sun Sep 15 02:10:57 +0000 2024,1,@ohmy_beautybank sejauh ini azarine calm my ac...,0,0,0,yoonginin
1421,Sun Sep 15 01:57:58 +0000 2024,0,@bynbiiys_blight Aku dari dulu nunggu yang wak...,0,1,0,Jenowsiee


In [13]:
#Melihat data yang duplikat
dataset['full_text'].duplicated().sum()

5

In [14]:
#Melihat missing value
dataset.isna().sum()

created_at        0
favorite_count    0
full_text         0
quote_count       0
reply_count       0
retweet_count     0
username          0
dtype: int64

In [16]:
slang_df = pd.read_csv("D:/Github/analisis-sosial-media/colloquial-indonesian-lexicon.csv")  # Kamus slang dengan kolom 'slang' dan 'formal
slang_df = slang_df.drop(columns=['In-dictionary', 'context', 'category1', 'category2', 'category3'])
print(slang_df)

          slang         formal
0          woww            wow
1         aminn           amin
2           met        selamat
3        netaas        menetas
4       keberpa       keberapa
...         ...            ...
15001  gataunya  enggak taunya
15002      gtau     enggak tau
15003     gatau     enggak tau
15004     fans2        fan-fan
15005   gaharus   enggak harus

[15006 rows x 2 columns]


In [19]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Najma Akmalina
[nltk_data]     Ni'ma\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [20]:
# Pembersihan Teks
def clean_text(text):
    # Lowercasing
    text = text.lower()
    # Hapus URL
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Hapus Mention dan Hashtag
    text = re.sub(r'@\w+|#\w+', '', text)
    # Hapus Angka dan Simbol
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Hapus White Spaces Berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

dataset['full_text'] = dataset['full_text'].apply(clean_text)

# Normalisasi
# Membuat kamus slang dari dataframe
slang_dict = pd.Series(slang_df.formal.values, index=slang_df.slang).to_dict()

def normalize_text(text):
    words = text.split()
    normalized_words = [slang_dict[word] if word in slang_dict else word for word in words]
    return ' '.join(normalized_words)

dataset['full_text'] = dataset['full_text'].apply(normalize_text)

# Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()
dataset['full_text'] = dataset['full_text'].apply(lambda x: stemmer.stem(x))

# Tokenization
dataset['tokens'] =dataset['full_text'].apply(lambda x: word_tokenize(x))

# Stop Words Removal
stop_words = set(stopwords.words('indonesian'))
dataset['tokens'] = dataset['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# 6. Handling Negation (Opsional)
def handle_negation(tokens):
    negation_words = {'tidak', 'bukan', 'belum', 'jangan'}
    processed_tokens = []
    skip_next = False
    for i, word in enumerate(tokens):
        if skip_next:
            skip_next = False
            continue
        if word in negation_words and i + 1 < len(tokens):
            processed_tokens.append(word + '_' + tokens[i + 1])
            skip_next = True
        else:
            processed_tokens.append(word)
    return processed_tokens

dataset['tokens'] = dataset['tokens'].apply(handle_negation)


# Contoh output
dataset

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - "C:\\Users\\Najma Akmalina Ni'ma/nltk_data"
    - 'e:\\APK\\anaconda3\\nltk_data'
    - 'e:\\APK\\anaconda3\\share\\nltk_data'
    - 'e:\\APK\\anaconda3\\lib\\nltk_data'
    - "C:\\Users\\Najma Akmalina Ni'ma\\AppData\\Roaming\\nltk_data"
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


,created_at,favorite_count,full_text,quote_count,reply_count,retweet_count,username,tokens
0,Mon Sep 09 23:49:15 +0000 2024,0,noo turut yang bagus cuma moistnya saja hiks v...,0,1,0,natashaless_,"[noo, bagus, moistnya, hiks, vit, sih, mending..."
1,Mon Sep 09 23:36:22 +0000 2024,0,kupon kilat tokopedia jam rekomendasi toko gua...,0,1,0,MSrwf,"[kupon, kilat, tokopedia, jam, rekomendasi, to..."
2,Mon Sep 09 23:33:29 +0000 2024,0,cek azarine calm my acne sunscreen moisturiser...,0,0,0,YZ_Fashion,"[cek, azarine, calm, my, acne, sunscreen, mois..."
3,Mon Sep 09 23:25:27 +0000 2024,1,sunscreen biru azarine yang sekaligus body lot...,0,3,0,ginnobaraaa,"[sunscreen, biru, azarine, body, lotion, wewan..."
4,Mon Sep 09 22:06:47 +0000 2024,0,coba azarine yang ijo aku dulu cocok pakai itu,0,0,0,apri_i002,"[coba, azarine, ijo, cocok, pakai]"
...,...,...,...,...,...,...,...,...
595,Wed Sep 04 14:53:24 +0000 2024,0,iyup sama azarine,0,1,0,jeocalist,"[iyup, azarine]"
596,Wed Sep 04 14:50:35 +0000 2024,0,aku nemu promo tarik di lazada nih yuk coba li...,0,0,0,riskaamaa_,"[nemu, promo, tarik, lazada, nih, yuk, coba, l..."
597,Wed Sep 04 14:50:17 +0000 2024,0,ss azarine memang the best sih,0,0,0,onenisjh,"[ss, azarine, the, best, sih]"
598,Wed Sep 04 14:48:09 +0000 2024,0,toner azarine favorite banget,0,0,0,handsaniteezer,"[toner, azarine, favorite, banget]"


In [ ]:
shopee_links = dataset[dataset['full_text'].str.contains('bagus', na=False)]
shopee_links

,created_at,favorite_count,full_text,quote_count,reply_count,retweet_count,username,tokens
0,Mon Sep 09 23:49:15 +0000 2024,0,noo turut yang bagus cuma moistnya saja hiks v...,0,1,0,natashaless_,"[noo, bagus, moistnya, hiks, vit, sih, mending..."
19,Mon Sep 09 13:59:43 +0000 2024,1,yang aqua rich bagus bau alkohol sudah enggak ...,0,1,0,sukaskincarean,"[aqua, rich, bagus, bau, alkohol, strong, kaya..."
24,Mon Sep 09 13:06:19 +0000 2024,1,kalo enggak pepet banget aku akan hindar merk ...,0,2,0,intoxicatae,"[kalo, pepet, banget, hindar, merk, pakai, ski..."
26,Mon Sep 09 12:35:22 +0000 2024,0,azarine memang bagus sih,0,0,0,cryingbuby,"[azarine, bagus, sih]"
33,Mon Sep 09 12:08:50 +0000 2024,3,azarine bagus banget label kuning karena beli ...,0,3,0,another_xrl,"[azarine, bagus, banget, label, kuning, beli, ..."
52,Mon Sep 09 08:22:49 +0000 2024,0,dari semua ss itu paling bagus yang mana kak s...,0,0,0,katanyasihgitu7,"[ss, bagus, kak, saran, ss, bagus, oily, acne,..."
65,Mon Sep 09 06:20:59 +0000 2024,0,azarine kata teman ku bagus kalo hanasui aku p...,0,1,0,heart2lou,"[azarine, teman, ku, bagus, kalo, hanasui, pak..."
68,Mon Sep 09 05:38:18 +0000 2024,0,mau yang aga murah azarine kalo yang bagus ya ...,0,0,0,txtdariuwa,"[aga, murah, azarine, kalo, bagus, ya, avoskin]"
113,Sun Sep 08 23:06:17 +0000 2024,0,beda ini sama azarine apa ya lebih bagus ini a...,0,0,0,kokoolatemin_,"[beda, azarine, ya, bagus, azarine, ya, bingun..."
117,Sun Sep 08 18:46:23 +0000 2024,0,azarine yang calm my acne bagus,0,0,0,brekoliforlife,"[azarine, calm, my, acne, bagus]"
